In [2]:
import jieba, os, re
from gensim import corpora, models, similarities

def stopwordslist():
  stopwords = [line.strip() for line in open('./stopwords.txt', encoding='utf-8').readlines()]
  return stopwords

# stopwordslist()


def seg_depart(sentence):
  '''
  切分单个句子成分词, sentence输的句子
  '''
  jieba.load_userdict("../socialnet/name.txt")
  sentence_depart = jieba.cut(sentence.strip())
  stopwords = stopwordslist()
  outstr = ''
  
  for word in sentence_depart:
    if word not in stopwords:
      outstr += word
      outstr += " "
      
  return outstr


def generate_chapeters_from_book(file):
  '''
  从整本书中生成每一章
  '''
  chapter = ""
  lines = file.readlines()
  chapter_token = u"第[\u4E00-\u9FA5]+章"
  pattern = re.compile(chapter_token)
  num = 0 #章节数
  if not os.path.exists("./book"):
    os.makedirs("./book")
  for line in lines:
    if pattern.search(line) is None:
      chapter += line
    else:
      if num == 0:
        num += 1
        continue
      with open("./book/第{}章.txt".format(num),mode='w',encoding='utf-8') as f:
        f.write(chapter)
        
      chapter = ""
      num += 1
      
  with open("./book/第{}章.txt".format(num),mode='w',encoding='utf-8') as f:
    f.write(chapter)


with open("./book.txt",encoding='utf-8') as file:
  generate_chapeters_from_book(file)
    
    
def tokenize_chapter(chapter, chapter_name):
  '''
  对每个章节进行分词, 并保存到对应的文件中
  '''
  output_file = "分词_" + chapter_name
  if not os.path.exists("./分词"):
    os.makedirs("./分词")
  with open("./分词/"+output_file,mode='w',encoding='utf-8') as f:
    for line in chapter:
      line = re.sub(u'[^\u4e00-\u9fa5]+','',line)
      line_seg = seg_depart(line.strip())
      f.write(line_seg.strip()+'\n')
  
  print("=> 完成对{}内容的分词".format(chapter_name))

def tokenize():
  file_names = list(os.listdir("./book"))
  for file_name in file_names:
    with open("./book/"+file_name, mode='r', encoding='utf-8') as chapter:
      tokenize_chapter(chapter.readlines(), file_name)

  print("=> 分词完成")

tokenize()
  

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ljj12\Temp\jieba.cache
Loading model cost 1.805 seconds.
Prefix dict has been built successfully.


=> 完成对第100章.txt内容的分词
=> 完成对第101章.txt内容的分词
=> 完成对第102章.txt内容的分词
=> 完成对第103章.txt内容的分词
=> 完成对第104章.txt内容的分词
=> 完成对第105章.txt内容的分词
=> 完成对第106章.txt内容的分词
=> 完成对第107章.txt内容的分词
=> 完成对第108章.txt内容的分词
=> 完成对第109章.txt内容的分词
=> 完成对第10章.txt内容的分词
=> 完成对第110章.txt内容的分词
=> 完成对第111章.txt内容的分词
=> 完成对第112章.txt内容的分词
=> 完成对第113章.txt内容的分词
=> 完成对第114章.txt内容的分词
=> 完成对第115章.txt内容的分词
=> 完成对第116章.txt内容的分词
=> 完成对第117章.txt内容的分词
=> 完成对第118章.txt内容的分词
=> 完成对第119章.txt内容的分词
=> 完成对第11章.txt内容的分词
=> 完成对第120章.txt内容的分词
=> 完成对第121章.txt内容的分词
=> 完成对第122章.txt内容的分词
=> 完成对第123章.txt内容的分词
=> 完成对第124章.txt内容的分词
=> 完成对第125章.txt内容的分词
=> 完成对第126章.txt内容的分词
=> 完成对第127章.txt内容的分词
=> 完成对第128章.txt内容的分词
=> 完成对第129章.txt内容的分词
=> 完成对第12章.txt内容的分词
=> 完成对第130章.txt内容的分词
=> 完成对第131章.txt内容的分词
=> 完成对第132章.txt内容的分词
=> 完成对第133章.txt内容的分词
=> 完成对第134章.txt内容的分词
=> 完成对第135章.txt内容的分词
=> 完成对第136章.txt内容的分词
=> 完成对第137章.txt内容的分词
=> 完成对第138章.txt内容的分词
=> 完成对第139章.txt内容的分词
=> 完成对第13章.txt内容的分词
=> 完成对第140章.txt内容的分词
=> 完成对第141章.txt内容的分词
=> 完成对第142章.txt内容的分词
=> 完成对第143章.txt内容

In [8]:
dir = "./分词/"
tokenized_filenames = os.listdir(dir)

with open("./compose_books.txt",mode='r',encoding='utf-8') as tokenized_chapter:
  tokenized_lines = []
  for line in tokenized_chapter:
    line = [word.strip() for word in line.split(' ')]
    tokenized_lines.append(line)
    
  # print(tokenized_line) 
  '''构建词频矩阵，训练LDA模型'''
  dictionary = corpora.Dictionary(tokenized_lines) 
  # print(dictionary)
  corpus = [dictionary.doc2bow(text) for text in tokenized_lines]
  # print(corpus)
  lda = models.LdaModel(corpus=corpus,id2word=dictionary,num_topics=10)
  topic_list = lda.print_topics(10)
  print("10个主题的单词分布为：\n")
  for topic in topic_list:
    print(topic)


10个主题的单词分布为：

(0, '0.016*"说" + 0.007*"里" + 0.006*"事" + 0.004*"少平" + 0.004*"中" + 0.003*"想" + 0.003*"少安" + 0.003*"象" + 0.003*"两个" + 0.003*"走"')
(1, '0.011*"说" + 0.005*"里" + 0.004*"田福堂" + 0.003*"事" + 0.003*"两个" + 0.003*"双水村" + 0.003*"黄原" + 0.003*"中" + 0.003*"少平" + 0.003*"生活"')
(2, '0.012*"说" + 0.006*"里" + 0.004*"少平" + 0.003*"事" + 0.003*"走" + 0.003*"中" + 0.003*"田福军" + 0.002*"想" + 0.002*"公社" + 0.002*"生活"')
(3, '0.012*"说" + 0.007*"里" + 0.005*"少平" + 0.004*"中" + 0.004*"走" + 0.003*"两个" + 0.003*"想" + 0.003*"少安" + 0.003*"象" + 0.003*"事"')
(4, '0.011*"说" + 0.005*"里" + 0.004*"少平" + 0.004*"事" + 0.003*"中" + 0.003*"生活" + 0.003*"象" + 0.003*"走" + 0.003*"两个" + 0.002*"吃"')
(5, '0.011*"说" + 0.006*"里" + 0.004*"两个" + 0.004*"事" + 0.004*"少平" + 0.003*"走" + 0.003*"中" + 0.003*"黄原" + 0.002*"少安" + 0.002*"想"')
(6, '0.011*"说" + 0.007*"里" + 0.004*"吃" + 0.004*"走" + 0.003*"中" + 0.003*"想" + 0.003*"两个" + 0.003*"少平" + 0.003*"事" + 0.003*"双水村"')
(7, '0.009*"里" + 0.008*"说" + 0.004*"走" + 0.003*"少平" + 0.003*"事" + 0.003*"两个" + 0.